In [1]:
import pandas as pd
import numpy as np
from sklearn import ensemble, preprocessing, metrics
from sklearn.metrics import f1_score
from joblib import dump,load
from sklearn import model_selection
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import recall_score, precision_score
import networkx as nx
import os
import ast
import copy
import csv

In [3]:
dataset = pd.read_csv(r'./Labels_TimeSyscallSeqs.csv')
# print(len(dataset[4][1]))


In [4]:
dataset

,FILENAME,CLASS,BEH,FAM,SEQUENCE
0,ca970cc75ccdf7bd1959b970de1f81d682ea3af871aff6...,backdoor,server,mirai,"[('1659223215.715424', 'execve'), ('1659223215..."
1,699ffec8df22106524defd950bff835735051514a9b059...,unknown,unknown,mirai,"[('1662079445.156731', 'execve'), ('1662079445..."
2,94e883f37be5e6a2791b081e78c86804b427e90e55a172...,backdoor,server,mirai,"[('946684902.916742', 'execve'), ('946684903.7..."
3,6b8490cc0ea6b234de8b6529b9277ee0ccba22026b4689...,backdoor,ddos,unknown,"[('1970.5854504280915968', 'wait4'), ('1970.62..."
4,93a9669eea9b49258a1c4ca20b261561b6bd91b43643ad...,backdoor,ddos,mirai,"[('1970.-1293549660274688', 'read'), ('1970.-1..."
...,...,...,...,...,...
29216,0d3c90bbe475fff13feab71a0fa354b370816293e76bd1...,backdoor,server,mirai,"[('946684980.350152', 'execve'), ('946684981.6..."
29217,44dae951b8795d619aa703ec45d9ca2a9163bcfbc7e0e8...,unknown,unknown,mirai,"[('1661739044.775424', 'execve'), ('1661739044..."
29218,31cf4befa49497a5195671d5e9c861730599817d3a6c99...,backdoor,server,unknown,"[('1661842757.198039', 'execve'), ('1661842757..."
29219,4859747535ff9088e328535a667c54b42bee132678d492...,backdoor,ddos,mirai,"[('1660561355.452809', 'execve'), ('1660561355..."


In [5]:
# new_dataset = []
# cnt_mirai = 0
# cnt_unknown = 0
# drop_fam = ['metasploit','zbot','skeeyah','hiddad']
# drop_index = []
# dataset = dataset.drop(['CLASS', 'BEH'], axis=1)
# for i in range(len(dataset)):
#     if dataset['FAM'][i] == 'mirai':
#         if cnt_mirai > 50:
#             drop_index.append(i)
#         else:
#             cnt_mirai += 1
#     elif dataset['FAM'][i] == 'unknown':
#         if cnt_unknown > 50 :
#             drop_index.append(i)
#         else :
#             cnt_unknown += 1
#     elif dataset['FAM'][i] in drop_fam:
#         drop_index.append(i)
# dataset = dataset.drop(drop_index)

In [6]:
new_dataset = []
cnt_mirai = 0
cnt_unknown = 0
drop_fam = ['metasploit','zbot','skeeyah','hiddad']
drop_index = []
dataset = dataset.drop(['CLASS', 'BEH'], axis=1)
for i in range(len(dataset)):
    # if dataset['FAM'][i] == 'mirai':
    #     if cnt_mirai > 50:
    #         drop_index.append(i)
    #     else:
    #         cnt_mirai += 1
    # elif dataset['FAM'][i] == 'unknown':
    #     if cnt_unknown > 50 :
    #         drop_index.append(i)
    #     else :
    #         cnt_unknown += 1
    if dataset['FAM'][i] in drop_fam:
        drop_index.append(i)
dataset = dataset.drop(drop_index)

In [7]:
dataset = dataset.reset_index(drop = True)

In [8]:
y = np.array(dataset["FAM"])

In [9]:
syscall_list = []

for i in range(len(dataset)):
    tmp_syscall = []
    tmp = str(dataset["SEQUENCE"][i])
    tmp = tmp.replace('-','0')
    # print(i)
    tmp_seq = ast.literal_eval(tmp)
    end_t = float(tmp_seq[len(tmp_seq) - 1][0])
    start_t = float(tmp_seq[0][0])

    for j in range(len(tmp_seq)):
        try:
            time = float(tmp_seq[j][0])
            time -= start_t
            tmp_syscall.append([time,tmp_seq[j][1]])
        except:
            pass
    syscall_list.append(tmp_syscall)

In [10]:
syscallCategory = {
    'kernel' : ['rt_sigaction', 'uname', 'setgid', 'getpriority', 'time', 'clone', 'restart_syscall', 'sysinfo', 'prlimit64', 'geteuid', 'kill', 'umask', 'getppid', 'set_robust_list', 'setresuid', 'nanosleep', 'rt_sigprocmask', 'prctl', 'times', 'mmap', 'setsid', 'vfork', 'wait4', 'getuid', 'gettid', 'set_tid_address', 'fork', 'rt_sigsuspend', 'setpriority', 'ptrace', 'get_thread_area', 'exit', 'alarm', 'setpgid', 'setresgid', 'set_thread_area', 'sigaltstack', 'getrlimit', 'getpid', 'futex', 'setrlimit', 'getegid', 'tgkill', 'setuid', 'exit_group', 'clock_gettime', 'getgid', 'rt_sigtimedwait', 'setitimer', 'getpgrp', 'gettimeofday','mmap2','ni_syscall','sigreturn','waitpid'],

    'fs' : ['stat', 'lseek', 'readlinkat', 'chroot', 'sendfile', 'umount2', 'symlink', 'flock', 'dup2', 'getcwd', 'chdir', 'fstat', 'mount', 'rmdir', 'execve', 'mkdir', 'epoll_wait', 'openat', 'eventfd2', 'readv', 'rename', 'epoll_create1', 'fchmod', 'pipe', 'unlink', 'pipe2', 'fcntl', 'open', 'read', 'write', 'lstat', 'chmod', 'readlink', 'getdents64', 'utimes', 'ioctl', 'select', 'access', 'close', 'poll', 'getdents', 'epoll_ctl', 'ftruncate','_llseek','_newselect','fcntl64','fstat64','llseek','lstat64','renameat2','stat64'],

    'net' : ['accept', 'connect', 'sendto', 'shutdown', 'getsockname', 'getpeername', 'listen', 'socketpair', 'socket', 'setsockopt', 'getsockopt', 'recvfrom', 'recvmsg', 'bind','recv','send','sendfile64','socketcall'],

    'mm' : ['mprotect', 'brk', 'munmap', 'madvise'],

    'ipc' : ['shmdt', 'shmget'],

    'printk' : ['syslog'],

    'sched' : ['sched_getaffinity'],
    
    'HighFreq' : ['_newselect','close','connect','fcntl','get_thread_area','getsockopt','open','read','recv','recvfrom','rt_sigaction','rt_sigprocmask','sendto','socket','time'],

    'other' : [ 'getegid32','geteuid32','getgid32','getuid32','setgid32','setresuid32','setuid32','sysctl','ugetrlimit']
}

In [11]:
fam = {
    'kernel':0,
    'fs':1,
    'net':2,
    'mm':3,
    'ipc':4,
    'printk':5,
    'sched':6,
    'HighFreq':7,
    'other':8
}

In [12]:
def get_key(target):
    return_key = []
    for key,value in syscallCategory.items():
        if target in value:
            return_key.append(key)
            # return key
    if len(return_key) > 0:
        return return_key
    else:
        return ['other']

In [13]:
column, row = 16, 9

imgs = []

# print(step)
for i in range(len(syscall_list)):
    time_step = np.linspace(0.0,float(syscall_list[i][-1][0]),num=17)
    time_step = time_step[1:]
    k = 0
    step = [[0 for _ in range(row)] for _ in range(column)]
    # print(time_step)
    for j in range(len(syscall_list[i])):
        if syscall_list[i][j][0] <= time_step[k]:
            syscall_fam = get_key(syscall_list[i][j][1])
            for h in syscall_fam:
                step[k][fam[h]] += 1
        else:
            k += 1
            syscall_fam = get_key(syscall_list[i][j][1])
            for h in syscall_fam:
                step[k][fam[h]] += 1                        
    imgs.append(step)
    # print("*"*20)
    # print(step)
    # print(len(syscall_list[i]))
    
# print(step)
# print(step.sum())


In [14]:
imgs = np.array(imgs)

In [15]:
imgs.shape

(29213, 16, 9)

In [16]:
np.save("./img_data_all.npy",imgs)
np.save("./img_label_all.npy",y)

In [17]:
_y = np.load('./ML_security/img_label.npy',allow_pickle=True)
print(_y.shape)

FileNotFoundError: [Errno 2] No such file or directory: './ML_security/img_label.npy'

In [ ]:
d = {}
for i, fam in enumerate(dataset['FAM']):
    if not fam in d:
        d[fam] = 1
    else :
        d[fam] += 1
print(d)

In [ ]:
# 確定class種類
class_num = 0
for i in d.keys():
    if d[i] > 5:
        print(i," : ",d[i])
        class_num += 1
print("*"*20)
print("class num : ",class_num)

In [ ]:
'kernel' : ['rt_sigaction', 'uname', 'setgid', 'getpriority', 'time', 'clone', 'restart_syscall', 'sysinfo', 'prlimit64', 'geteuid', 'kill', 'umask', 'getppid', 'set_robust_list', 'setresuid', 'nanosleep', 'rt_sigprocmask', 'prctl', 'times', 'mmap', 'setsid', 'vfork', 'wait4', 'getuid', 'gettid', 'set_tid_address', 'fork', 'rt_sigsuspend', 'setpriority', 'ptrace', 'get_thread_area', 'exit', 'alarm', 'setpgid', 'setresgid', 'set_thread_area', 'sigaltstack', 'getrlimit', 'getpid', 'futex', 'setrlimit', 'getegid', 'tgkill', 'setuid', 'exit_group', 'clock_gettime', 'getgid', 'rt_sigtimedwait', 'setitimer', 'getpgrp', 'gettimeofday','mmap2','ni_syscall','sigreturn','waitpid'],

In [ ]:
'kernel_signal' : ['rt_sigaction', 'restart_syscall','kill', 'rt_sigprocmask','rt_sigsuspend','sigaltstack','tgkill','rt_sigtimedwait','sigreturn']
'kernel_sys' : ['uname', 'setgid', 'getpriority', 'sysinfo', 'prlimit64','geteuid','umask', 'getppid', 'setresuid','prctl', 'times','mmap','setsid','getuid', 'gettid', 'setpriority','setpgid', 'setresgid','getrlimit','getpid','setrlimit','getegid','setuid','getgid','getpgrp','mmap2',''ni_syscall'']
'kernel_time' : ['time','gettimeofday']
'kernel_timer' :['alarm']
'kernel_fork' : ['clone','vfork','set_tid_address', 'fork']
'kernel_futex' : ['set_robust_list','futex']
'kernel_hrtimer': ['nanosleep']
'kernel_exit' : ['wait4','exit','exit_group','waitpid']
'kernel_ptrace': ['ptrace']
'kernel_tls':['get_thread_area','set_thread_area']
'kernel_posix-timers' : ['clock_gettime']
'kernel_itimer' : ['setitimer']